# Initial SETUP

In [1]:
import os 

SETUP = {
    'CURR_DIR': os.getcwd(),
    'DEBUG': True,
    'DEBUG_EPISODES': 500,
    'EPISODES': 2000,
    'MAX_STEPS': 1000,
    'EPS_INI': 1.0, 
    'EPS_END': 1e-4,
    'EPS_DECAY': 0.995,
    'GAMMA': 1,
    'SEED':32,  
    'BUFFER_SIZE': int(1e5),
    'BATCH_SIZE': 64,
    'GAMMA': 0.99,
    'TAU': 1e-3,
    'LR': 5e-4,
    'UPDATE_EVERY': 4,
    'FC1_UNITS': 64,
    'FC2_UNITS': 64,
}

try:
  import google.colab
  SETUP['IN_COLAB'] = True
except: SETUP['IN_COLAB'] = False

SUBVERSION = "v1.0.0"
SPECIFIC_RUN = 'SimpleCNN'
VERSION_KEY = 'BananaNavigation'

SETUP['RUN-KEY'] = "_".join([SUBVERSION, SPECIFIC_RUN, VERSION_KEY]) + '_'

In [2]:
from pathlib import Path

if SETUP['IN_COLAB']:
    from google.colab import drive
    drive.mount('/content/gdrive')
    SETUP['PATH'] = Path("/content/gdrive/MyDrive/_Colab/RL-SmartAgent-BananaGame")
    !pip install optuna --quiet
    !cd '/content/gdrive/MyDrive/_Colab/RL-SmartAgent-BananaGame/python'; pip install .
else: SETUP['PATH'] = Path(".")

SETUP['BANANA_PATH'] = str(SETUP['PATH'] / 'Banana_Linux_NoVis/Banana.x86_64') if SETUP['IN_COLAB'] else str(SETUP['PATH'] / "Banana.app")
os.chdir(SETUP['PATH'])

if SETUP['DEBUG']: SETUP['EPISODES'] = SETUP['DEBUG_EPISODES']

# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [3]:
from unityagents import UnityEnvironment
import unityagents
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [4]:
SETUP['BANANA_PATH']

'Banana.app'

In [5]:
#env = default_registry["Banan"].make()

env = UnityEnvironment(file_name=SETUP['BANANA_PATH'])

Mono path[0] = '/Users/joao/Cloud/GDrive/_Colab/RL-SmartAgent-BananaGame/Banana.app/Contents/Resources/Data/Managed'
Mono config path = '/Users/joao/Cloud/GDrive/_Colab/RL-SmartAgent-BananaGame/Banana.app/Contents/MonoBleedingEdge/etc'


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [6]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

print(brain_name, "\n\n\n",brain)

BananaBrain 


 Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


### 2. Examine the State and Action Spaces

    The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [7]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [8]:
def random_step():
    env_info = env.reset(train_mode=False)[brain_name] # reset the environment
    state = env_info.vector_observations[0]            # get the current state
    score = 0                                          # initialize the score
    while True:
        action = np.random.randint(action_size)        # select an action
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = env_info.rewards[0]                   # get the reward
        done = env_info.local_done[0]                  # see if episode has finished
        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        #print(env_info.vector_observations)
        print(f'\rscore: {score:<5}',end='')
        if done: 
            print(f'\nend of episode {time.time()-start:.3f} seg') # exit loop if episode finished
            break

    print("Final Score: {}".format(score))
    env.close()

When finished, you can close the environment.

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [9]:
from agent import Agent
from collections import deque
import torch
import time

PASSING_SCORE = 13

def dqn(agent_params, n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995, verbose=True):
    agent = Agent(**agent_params)

    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name]
        #state = env.reset()
        state = env_info.vector_observations[0]
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        if verbose: print(f'\rEpisode {i_episode}\tAverage Score: {np.mean(scores_window):<4.2f}', end="")
        if i_episode % 100 == 0 and verbose: print()
        if np.mean(scores_window) >= PASSING_SCORE:
            if verbose: print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            if not SETUP['DEBUG']: torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            return i_episode
#    return np.where(np.array(scores) > PASSING_SCORE)[0][0] if len(np.where(np.array(scores) > PASSING_SCORE)[0]) > 0 else 999999
    return i_episode + (np.where(np.array(scores) > PASSING_SCORE)[0][0] if len(np.where(np.array(scores) > PASSING_SCORE)[0]) > 0 else 999999)

In [10]:
def objective(trial):
    gamma = trial.suggest_float("gamma", 0.85, 1, log=True)
    tau = trial.suggest_float("tau", 1e-6, 1e-1, log=True)
    lr = trial.suggest_float("lr", 1e-6, 1e-1, log=True)
    fc1_units = trial.suggest_categorical("fc1_units", [12,32,64,128,256,1024])
    fc2_units = trial.suggest_categorical("fc2_units", [12,32,64,128,256,1024])
    eps_end = trial.suggest_float("eps_end", 1e-6, 1e-1, log=True)
    eps_decay = trial.suggest_float("gamma", 0.85, 1, log=True)

    agent_params = {'state_size': state_size, 'action_size':action_size, 'seed':SETUP['SEED'],  
                    'BUFFER_SIZE': SETUP['BUFFER_SIZE'], 'BATCH_SIZE': SETUP['BATCH_SIZE'], 
                    'GAMMA': gamma, 'TAU': tau, 'LR': lr, 'UPDATE_EVERY': SETUP['UPDATE_EVERY'],
                    'fc1_units': fc1_units, 'fc2_units': fc2_units
                   }
#    print(agent_params)
 #   print(SETUP['EPISODES'], SETUP['MAX_STEPS'], SETUP['EPS_INI'],eps_end, eps_decay)
    score = dqn(agent_params, SETUP['EPISODES'], SETUP['MAX_STEPS'], SETUP['EPS_INI'],
               eps_end, eps_decay, verbose=False)
    print('finished study got', score)
    #if score == 99999: raise optuna.exceptions.TrialPruned()
    return score

In [ ]:
import optuna
from optuna.trial import TrialState


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, timeout=60*60*2)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2021-07-12 17:26:37,225] A new study created in memory with name: no-name-9f430f2f-e98b-4750-8e83-df9055e2e427
[I 2021-07-12 17:30:15,672] Trial 0 finished with value: 434.0 and parameters: {'gamma': 0.915788017940018, 'tau': 0.0890641567864362, 'lr': 0.0012442446822827895, 'fc1_units': 12, 'fc2_units': 32, 'eps_end': 2.4650961960412612e-05}. Best is trial 0 with value: 434.0.


finished study got 434


[I 2021-07-12 17:37:34,233] Trial 1 finished with value: 1000499.0 and parameters: {'gamma': 0.9168517716774827, 'tau': 3.1217691008992035e-06, 'lr': 0.008359994464797288, 'fc1_units': 256, 'fc2_units': 256, 'eps_end': 0.007975419338330774}. Best is trial 0 with value: 434.0.


finished study got 1000499


[I 2021-07-12 17:44:59,203] Trial 2 finished with value: 1000499.0 and parameters: {'gamma': 0.8900844311089697, 'tau': 2.034269578949832e-05, 'lr': 7.338621830143718e-06, 'fc1_units': 1024, 'fc2_units': 12, 'eps_end': 3.861054257280012e-05}. Best is trial 0 with value: 434.0.


...

finished study got 330


[I 2021-07-12 19:20:26,604] Trial 17 finished with value: 413.0 and parameters: {'gamma': 0.9382043092555059, 'tau': 0.0010335739249840937, 'lr': 0.00010705607069079874, 'fc1_units': 128, 'fc2_units': 128, 'eps_end': 0.004728535089557808}. Best is trial 16 with value: 330.0.


finished study got 413


[I 2021-07-12 19:23:29,657] Trial 18 finished with value: 355.0 and parameters: {'gamma': 0.9748704565566926, 'tau': 0.0020638075353569874, 'lr': 0.0003065748753448507, 'fc1_units': 12, 'fc2_units': 128, 'eps_end': 0.004523276470011786}. Best is trial 16 with value: 330.0.


finished study got 355


In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
#!pip install -U plotly>=4.0.0
import joblib
import optuna
study = joblib.load('study.pkl')
#study.trials # error
optuna.visualization.plot_optimization_history(study)
#import plotly
#plotly.version
#!python3 -m pip install plotly

#joblib.dump(study, 'study.pkl')


### New heading

In [ ]:
import matplotlib.pyplot as plt
print(f'First time reaches target: {np.where(np.array(scores) > PASSING_SCORE)[0][0]}; maximum score: {max(scores)}')
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()